In [47]:
from urllib.request import urlopen

In [54]:
rss = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109'
f = urlopen(rss)

In [ ]:
weather_xml = f.read().decode('utf-8')
weather_xml

In [56]:
from xml.etree import ElementTree

In [ ]:
root = ElementTree.fromstring(weather_xml)

for item in root.findall('channel/item/description/body/location/data'):
    # find() 메서드로 element 탐색, text 속성으로 값을 추출
    tm_ef = item.find('tmEf').text
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    print(f'일시 :  {tm_ef}, 최저기온: {tmn}, 최고기온: {tmx}, 날씨: {wf}')
    

In [29]:
import pandas as pd

In [30]:
# 1. 리스트 생성
# 2. for 파일 일기
#     데이터프레임 생성
#     데이터프레임 -> 리스트 추가(append)
# 3. pd.concat(리스트)
# 4. 인덱스 재생성

In [62]:
데이터프레임_리스트 = []
for item in root.findall('channel/item/description/body/location/data'):
    # find() 메서드로 element 탐색, text 속성으로 값을 추출
    tm_ef = item.find('tmEf').text
    tmn = item.find('tmn').text
    tmx = item.find('tmx').text
    wf = item.find('wf').text
    데이터프레임 = pd.DataFrame({
        '일시':[tm_ef],
        '최저기온':[tmn],
        '최고기온':[tmx],
        '날씨':[wf],
    })
    데이터프레임_리스트.append(데이터프레임)
날씨정보 = pd.concat(데이터프레임_리스트)
날씨정보  

,일시,최저기온,최고기온,날씨
0,2023-08-22 00:00,25,29,흐리고 비
0,2023-08-22 12:00,25,29,흐리고 비
0,2023-08-23 00:00,25,29,흐리고 비
0,2023-08-23 12:00,25,29,흐리고 비
0,2023-08-24 00:00,24,29,흐리고 비
...,...,...,...,...
0,2023-08-26 00:00,22,29,구름많음
0,2023-08-26 12:00,22,29,흐림
0,2023-08-27 00:00,21,30,흐림
0,2023-08-28 00:00,22,29,흐림


In [65]:
날씨정보.reset_index(drop=True, inplace=True)
날씨정보

,일시,최저기온,최고기온,날씨
0,2023-08-22 00:00,25,29,흐리고 비
1,2023-08-22 12:00,25,29,흐리고 비
2,2023-08-23 00:00,25,29,흐리고 비
3,2023-08-23 12:00,25,29,흐리고 비
4,2023-08-24 00:00,24,29,흐리고 비
...,...,...,...,...
450,2023-08-26 00:00,22,29,구름많음
451,2023-08-26 12:00,22,29,흐림
452,2023-08-27 00:00,21,30,흐림
453,2023-08-28 00:00,22,29,흐림


In [64]:
type(날씨정보)

pandas.core.frame.DataFrame

In [8]:
날씨정보.to_csv('날씨정보.csv')

In [9]:
엑셀 = pd.ExcelWriter('날씨정보.xlsx')
날씨정보.to_excel(엑셀, '.', index=False )
엑셀.save()

In [11]:
날씨정보.to_json('날씨정보.json')

In [12]:
import sqlite3
from pandas.io import sql
import os

In [13]:
with sqlite3.connect(os.path.join('.','sqliteDB')) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
    try:
        날씨정보.to_sql(name = 'WEATHER_INFO', con = con, index = False, if_exists='append') 
        #if_exists : {'fail', 'replace', 'append'} default : fail
    except Exception as e:
        print(str(e))
    
    query = 'SELECT * FROM WEATHER_INFO'
    데이터프레임1 = pd.read_sql(query, con = con)

In [14]:
데이터프레임1

,일시,최저기온,최고기온,날씨
0,2020-06-25 00:00,21,26,흐리고 비
1,2020-06-25 12:00,21,26,흐리고 비
2,2020-06-26 00:00,21,29,흐리고 비
3,2020-06-26 12:00,21,29,구름많음
4,2020-06-27 00:00,22,29,구름많음
...,...,...,...,...
528,2020-06-29 00:00,23,26,흐리고 비
529,2020-06-29 12:00,23,26,흐리고 비
530,2020-06-30 00:00,22,26,흐리고 비
531,2020-07-01 00:00,22,26,흐리고 비


In [ ]:
엑셀 = pd.ExcelWriter('날씨정보2.xlsx')
데이터프레임1.to_excel(엑셀, '.', index=False )
엑셀.save()

In [ ]:
df = pd.read_excel('날씨정보2.xlsx')

In [ ]:
df